In [4]:
from shapely.geometry import Point
import seaborn as sns
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

sns.set_style("dark")
#this is geospatial workbook - it might not like seaborn? getting error that it's not found

In [5]:
income_raw=pd.read_excel('../data/income_zip.xlsx')

In [39]:
income_raw.head(3)

,B19001,Household Income in the Past 12 Months (In 2019 Inflation-adjusted Dollars),Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84
0,NaN,37013,NaN,37015,NaN,37027,NaN,37062,NaN,37064,...,37221,NaN,37228,NaN,37240,NaN,37243,NaN,37246,NaN
1,NaN,Value,Error,Value,Error,Value,Error,Value,Error,Value,...,Value,Error,Value,Error,Value,Error,Value,Error,Value,Error
2,Total:,35597,617,6673,279,19950,382,4040,188,21640,...,17959,354,1051,240,0,12,0,12,0,12


In [11]:
income_data = pd.read_excel("../data/income_zip.xlsx", skiprows = 1, header=[0, 1], sheet_name = 0)
income_data = income_data[income_data.columns.drop(list(income_data.filter(regex='Error')))]
income_data.columns = income_data.columns.droplevel(1)
income_clean = income_data.set_index('Unnamed: 0_level_0').T.reset_index().rename({"index": "zipcode"}, axis = 1)



In [26]:
income_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   zipcode               42 non-null     object
 1   Total:                42 non-null     int64 
 2   Less than $10,000     42 non-null     int64 
 3   $10,000 to $14,999    42 non-null     int64 
 4   $15,000 to $19,999    42 non-null     int64 
 5   $20,000 to $24,999    42 non-null     int64 
 6   $25,000 to $29,999    42 non-null     int64 
 7   $30,000 to $34,999    42 non-null     int64 
 8   $35,000 to $39,999    42 non-null     int64 
 9   $40,000 to $44,999    42 non-null     int64 
 10  $45,000 to $49,999    42 non-null     int64 
 11  $50,000 to $59,999    42 non-null     int64 
 12  $60,000 to $74,999    42 non-null     int64 
 13  $75,000 to $99,999    42 non-null     int64 
 14  $100,000 to $124,999  42 non-null     int64 
 15  $125,000 to $149,999  42 non-null     int6

In [16]:
income_clean.head(50)

Unnamed: 0_level_0,zipcode,Total:,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more"
0,37013,35597,1424,1023,1197,1751,1647,1685,1748,1937,1748,3956,4510,5507,3547,1703,1212,1002
1,37015,6673,421,256,115,415,365,452,350,277,323,719,903,903,639,294,157,84
2,37027,19950,460,173,246,250,254,250,316,538,472,597,1235,1861,2167,1688,2837,6606
3,37062,4040,119,122,138,140,123,124,121,216,140,329,569,645,468,342,286,158
4,37064,21640,771,302,623,512,609,630,721,495,567,1479,1784,2293,2338,2265,2394,3857
5,37072,12533,534,412,456,674,687,506,518,568,544,1095,1311,2186,1120,784,652,486
6,37073,5425,233,151,169,236,290,325,282,184,288,549,557,698,620,262,399,182
7,37075,24275,743,422,553,687,682,986,1058,948,1011,2052,3003,3497,2271,2203,2004,2155
8,37076,17389,698,405,626,496,946,740,912,662,918,1445,2146,2648,1709,1167,1193,678
9,37080,3012,112,119,254,131,188,107,225,101,92,264,262,384,263,161,161,188


## Bring in the populations

In [18]:
pop_raw=pd.read_csv('../data/zipcode_population.csv')

In [38]:
pop_raw.head(3)

,number,zipcode,"lat,long",city,pop
0,1,37916,"35.953657, -83.933148","Knoxville, Tennessee","11,672"
1,2,37614,"36.301650, -82.370809","Johnson City, Tennessee","2,036"
2,3,37212,"36.133973, -86.801471","Nashville, Tennessee","18,547"


In [35]:
pop_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   number    610 non-null    int64 
 1   zipcode   610 non-null    object
 2   lat,long  610 non-null    object
 3   city      610 non-null    object
 4   pop       610 non-null    object
dtypes: int64(1), object(4)
memory usage: 24.0+ KB


In [ ]:
#make the zipcode an object to match income
pop_raw['zipcode']=pop_raw['zipcode'].astype(object)

In [23]:
#match the column names
pop_raw.rename(columns = {'zip':'zipcode'}, inplace=True)

In [37]:
pop_raw.head(20)

,number,zipcode,"lat,long",city,pop
0,1,37916,"35.953657, -83.933148","Knoxville, Tennessee","11,672"
1,2,37614,"36.301650, -82.370809","Johnson City, Tennessee","2,036"
2,3,37212,"36.133973, -86.801471","Nashville, Tennessee","18,547"
3,4,37351,"35.104194, -85.264615","Lupton City, Tennessee",270
4,5,37744,"36.182885, -82.740218","Greeneville, Tennessee",372
5,6,38105,"35.150642, -90.033969","Memphis, Tennessee","8,976"
6,7,38104,"35.132422, -90.004640","Memphis, Tennessee","24,727"
7,8,38115,"35.053329, -89.862806","Memphis, Tennessee","40,401"
8,9,38114,"35.096438, -89.985644","Memphis, Tennessee","34,120"
9,10,38111,"35.110521, -89.943501","Memphis, Tennessee","45,077"


##we need merge the income dataframe with the income dataframe, and we can do a join on the income zipcodes in order to include only davidson county

In [32]:
cases_income = pd.merge(income_clean,pop_raw, on=['zipcode'], how='left')

In [36]:
cases_income.head()

,zipcode,Total:,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999",...,"$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more",number,"lat,long",city,pop
0,37013,35597,1424,1023,1197,1751,1647,1685,1748,1937,...,4510,5507,3547,1703,1212,1002,NaN,NaN,NaN,NaN
1,37015,6673,421,256,115,415,365,452,350,277,...,903,903,639,294,157,84,NaN,NaN,NaN,NaN
2,37027,19950,460,173,246,250,254,250,316,538,...,1235,1861,2167,1688,2837,6606,NaN,NaN,NaN,NaN
3,37062,4040,119,122,138,140,123,124,121,216,...,569,645,468,342,286,158,NaN,NaN,NaN,NaN
4,37064,21640,771,302,623,512,609,630,721,495,...,1784,2293,2338,2265,2394,3857,NaN,NaN,NaN,NaN
